In [ ]:
model_name = "finbert_fr_gr"

In [ ]:
import wandb

wandb.login()

In [1]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'data/train_subset.csv', 'eval': 'data/eval_subset.csv'})

In [2]:
from utils import transform_labels
dataset = dataset.map(transform_labels)

In [3]:
# let's train a Distilbert model

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')

# let's tokenize the data for the model to be able to understand
def tokenize_data(example):
    return tokenizer(example['Sentence'], padding='max_length')    

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
dataset = dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForSequenceClassification

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", num_labels=3)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [6]:
# the default batch size for training arguments
batch_size = 8

# set number of epochs
number_of_epochs = 7
# let set the logging steps
logging_steps = len(dataset['train']) // batch_size # it should log each batch 

steps = (len(dataset['train']) / batch_size) * number_of_epochs
warmup_steps = int(0.2 * steps)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs, 
                                  load_best_model_at_end=True,
                                  evaluation_strategy='steps', 
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir=model_name,
                                  report_to='wandb',
                                  )

e:\repo\DistilBERTFinancialSentiment\.venv\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['eval'].shuffle(seed=10)

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [10]:
# Launch the learning process: training
# Load model from checkpoint
trainer.train()

Step,Training Loss,Validation Loss
500,1.114400,0.893000
1000,0.887200,0.896377
1500,0.743600,1.052437
2000,0.296200,1.392828
2500,0.168600,1.643464


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

TrainOutput(global_step=2800, training_loss=0.5384776483263288, metrics={'train_runtime': 637.9118, 'train_samples_per_second': 35.115, 'train_steps_per_second': 4.389, 'total_flos': 5893740557107200.0, 'train_loss': 0.5384776483263288, 'epoch': 7.0})

In [11]:
from utils import compute_metrics

trainer_eval = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [12]:
trainer_eval.evaluate()

Trainer is attempting to log a value of "{'accuracy': 0.6675}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.6650567577969442}" of type <class 'dict'> for key "eval/f1score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 1.714552879333496,
 'eval_model_preparation_time': 0.0026,
 'eval_accuracy': {'accuracy': 0.6675},
 'eval_f1score': {'f1': 0.6650567577969442},
 'eval_runtime': 8.9034,
 'eval_samples_per_second': 89.854,
 'eval_steps_per_second': 11.232}

In [ ]:
model.save_pretrained(model_name)
model.push_to_hub(f'nojedag/{model_name}')

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/finbert_fr_gr/commit/e508836911bdd9a47ea1e59c0d059bd5c7106d17', commit_message='Upload BertForSequenceClassification', commit_description='', oid='e508836911bdd9a47ea1e59c0d059bd5c7106d17', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/finbert_fr_gr', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/finbert_fr_gr'), pr_revision=None, pr_num=None)

In [16]:
trainer.push_to_hub()
trainer_eval.push_to_hub()

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

events.out.tfevents.1742653693.NOJEDA-PC.21160.0:   0%|          | 0.00/8.55k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1742654550.NOJEDA-PC.26288.0:   0%|          | 0.00/8.45k [00:00<?, ?B/s]

events.out.tfevents.1742654045.NOJEDA-PC.21160.1:   0%|          | 0.00/360 [00:00<?, ?B/s]

events.out.tfevents.1742655197.NOJEDA-PC.26288.1:   0%|          | 0.00/360 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/fine-tuned-distilbert-base-uncased/commit/147b03d5304980c361236bcd5786ced277927ce5', commit_message='End of training', commit_description='', oid='147b03d5304980c361236bcd5786ced277927ce5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/fine-tuned-distilbert-base-uncased', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/fine-tuned-distilbert-base-uncased'), pr_revision=None, pr_num=None)